## Setup

In [12]:
import tensorflow as tf
import tensorflow_hub as hub

from albumentations.pytorch import ToTensorV2
import albumentations as A

from imutils import paths
from tqdm.notebook import tqdm
import pickle
import json
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt

## Utilities

In [2]:
with open("imagenet_class_index.json", "r") as read_file:
    imagenet_labels = json.load(read_file)
    
MAPPING_DICT = {}
LABEL_NAMES = {}
for label_id in list(imagenet_labels.keys()):
    MAPPING_DICT[imagenet_labels[label_id][0]] = int(label_id)
    LABEL_NAMES[int(label_id)] = imagenet_labels[label_id][1]

IMAGENET_VAL_PATHS = list(paths.list_images("val"))
IMAGENET_VAL_PATHS_1k = np.random.choice(IMAGENET_VAL_PATHS, 1000)

f = open("IMAGENET_VAL_PATHS_1k.pkl", "wb")
f.write(pickle.dumps(IMAGENET_VAL_PATHS_1k))
f.close()

In [3]:
def preprocess(image_path, image_size=224):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, 3)
    image = tf.image.convert_image_dtype(image, tf.float32)  
    image = tf.image.resize(image, (image_size, image_size))
    image = tf.expand_dims(image, 0)
    return image

def preprocess_cutout(image_path, factor, image_size=224):
    total_area = image_size * image_size
    area = total_area * factor
    
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, 3)
    image = tf.image.resize(image, (image_size, image_size))
    image = tf.cast(image, tf.uint8)
   
    transform = A.Compose([
        A.Cutout(num_holes=1, max_h_size=int(np.sqrt(area)), 
                      max_w_size=int(np.sqrt(area)), 
                      always_apply=True,
                      p=1.)
    ])
    
    image = transform(image=image.numpy())["image"]
    image = tf.convert_to_tensor(image)
    image = tf.image.convert_image_dtype(image, tf.float32)  
    image = tf.expand_dims(image, 0)
    return image

## Evaluate without CutOut

In [4]:
BIT_URL = "https://tfhub.dev/google/bit/m-r101x3/ilsvrc2012_classification/1"
imagenet_module = tf.keras.Sequential([hub.KerasLayer(BIT_URL)])

In [13]:
def evaluate(factor=None):
    correct_image_paths = []

    for image_path in tqdm(IMAGENET_VAL_PATHS_1k):
        if factor:
            image = preprocess_cutout(image_path, factor)
        else:
            image = preprocess(image_path)
        label_idx = MAPPING_DICT[image_path.split("/")[1]]
        logits = imagenet_module.predict(image)
        prob = tf.nn.softmax(imagenet_module.predict(image), 1)
        prediction = tf.math.argmax(prob, 1).numpy().tolist()[0]

        if label_idx == prediction:
            correct_image_paths.append(image_path)

    print(f"Total corrects: {len(correct_image_paths)} out of {len(IMAGENET_VAL_PATHS_1k)}")
    return len(correct_image_paths) / len(IMAGENET_VAL_PATHS_1k)

In [14]:
print(evaluate())

  0%|          | 0/1000 [00:00<?, ?it/s]

Total corrects: 790 out of 1000
0.79


## Evaluate with CutOut at varying levels

In [15]:
factors = [0.05, 0.1, 0.2, 0.5]
factors_dict = {}

for factor in factors:
    top_1_acc = evaluate(factor)
    factors_dict[factor] = top_1_acc
    print(f"{factor}: {top_1_acc}")

  0%|          | 0/1000 [00:00<?, ?it/s]

Total corrects: 760 out of 1000
0.05: 0.76


  0%|          | 0/1000 [00:00<?, ?it/s]

Total corrects: 750 out of 1000
0.1: 0.75


  0%|          | 0/1000 [00:00<?, ?it/s]

Total corrects: 724 out of 1000
0.2: 0.724


  0%|          | 0/1000 [00:00<?, ?it/s]

Total corrects: 520 out of 1000
0.5: 0.52
